# Where To Open A New Coffee Shop In Toronto?

**Introduction:** An International Coffee chain is looking to expand to Canada and wants to open its first location in Toronto. It wants to ensure that its first location is a success, and is looking for an area that fits the following criteria:

- **Large customer base:** Densely populated area with many residents, offices and schools**
- **Underdeveloped coffee scene**: Not many existing coffee shops in the area

This analysis will aim to find Toronto neighbourhoods that satisfy these criteria

**Data**
This analysis will be performed by integrating the following data sets:
- Neighbourhood information for the City of Toronto that will be obtained from Wikipedia
- Venue related information for the City of Toronto that will be obtained from the Foursquare API

The data will be converted into a final pandas data frame for analysis, and Folium will be used to visualize it. Finally, k-means clustering will be used to discover neighbourhoods that are best suited to opening a new coffee shop 